In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)

from collections import OrderedDict

import sys
sys.path.append("..")
import utilities, ontology

# Load brain and text data

In [2]:
act_bin = utilities.load_coordinates().astype(float)
print("Document N={}, Structure N={}".format(act_bin.shape[0], act_bin.shape[1]))

Document N=18155, Structure N=114


In [3]:
version = 190325
dtm_bin = utilities.load_doc_term_matrix(version=version, binarize=True)
print("Document N={}, Term N={}".format(dtm_bin.shape[0], dtm_bin.shape[1]))

Document N=18155, Term N=4107


In [4]:
frameworks = ["data-driven", "rdoc", "dsm"]
lists, circuits = {}, {}
for fw in frameworks:
    lists[fw], circuits[fw] = utilities.load_framework(fw, suffix=utilities.suffix[fw])

In [5]:
scores = {fw: utilities.score_lists(lists[fw], dtm_bin, label_var="DOMAIN") for fw in frameworks}

In [6]:
words = []
for fw in frameworks:
    words += list(lists[fw]["TOKEN"])
words = sorted(list(set(words)))
structures = list(act_bin.columns)
print("Term N={}, Structure N={}".format(len(words), len(structures)))

Term N=271, Structure N=114


In [7]:
domains = {fw: list(OrderedDict.fromkeys(lists[fw]["DOMAIN"])) for fw in frameworks}

In [8]:
pmids = act_bin.index.intersection(scores["rdoc"].index).intersection(scores["dsm"].index)
len(pmids)

18155

In [9]:
for fw in frameworks:
    scores[fw] = scores[fw].loc[pmids]

In [10]:
dtm_bin = dtm_bin.loc[pmids, words]
act_bin = act_bin.loc[pmids, structures]

# Load frameworks

In [11]:
systems = {}
for fw in frameworks:
    fw_df = pd.DataFrame(0.0, index=words+structures, columns=domains[fw])
    for dom in domains[fw]:
        for word in lists[fw].loc[lists[fw]["DOMAIN"] == dom, "TOKEN"]:
            fw_df.loc[word, dom] = 1.0
        for struct in structures:
            fw_df.loc[struct, dom] = circuits[fw].loc[struct, dom]
    fw_df[fw_df > 0.0] = 1.0
    systems[fw] = fw_df

# Similarity of RDoC and data-driven systems

## Observed values

In [12]:
from scipy.spatial.distance import dice, cdist

In [13]:
def compute_sim_obs(fw):
    sims = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            sims.loc[k,r] = 1.0 - dice(systems["data-driven"][k], systems[fw][r])
    return sims

In [14]:
sims = compute_sim_obs("rdoc")
sims

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,0.301075,0.0674157,0,0.268041,0.292135,0
REWARD,0.136364,0.404762,0,0.0869565,0.119048,0
REACTION_TIME,0.42,0.520833,0.038835,0.0192308,0.166667,0
MANIPULATION,0,0,0.531915,0,0,0.606742
VISION,0.0444444,0,0.236559,0.234043,0.0232558,0.0681818
LANGUAGE,0.0842105,0,0,0.181818,0.153846,0.129032


## Null distribution

In [15]:
def compute_sim_null(fw, n_iter=1000):
    sims_null = np.empty((len(domains["data-driven"]), len(domains[fw]), n_iter))
    for n in range(n_iter):
        null = np.random.choice(words+structures, 
                                size=len(words+structures), replace=False)
        sims_null[:,:,n] = 1.0 - cdist(systems["data-driven"].loc[null].values.T, 
                                       systems[fw].values.T, metric="dice")
        if n % (float(n_iter) / 10.0) == 0:
            print("Iteration {}".format(n))
    return sims_null

In [16]:
sims_null = compute_sim_null("rdoc")

Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900


## False discovery rate

In [17]:
from statsmodels.stats.multitest import multipletests

In [18]:
def compute_sim_fdr(fw, sims, sims_null):
    n_iter = sims_null.shape[2]
    pvals = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for i, k in enumerate(domains["data-driven"]):
        for j, r in enumerate(domains[fw]):
            pvals.loc[k,r] = np.sum(sims_null[i,j,:] > sims.loc[k,r]) / float(n_iter)
    fdrs = multipletests(pvals.values.ravel(), method="fdr_bh")[1]
    fdrs = np.reshape(fdrs, pvals.shape)
    fdrs = pd.DataFrame(fdrs, index=domains["data-driven"], columns=domains[fw])
    return fdrs

In [19]:
fdrs = compute_sim_fdr("rdoc", sims, sims_null)
fdrs

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,0,1,1,0,0,1
REWARD,0.411429,0,1,1,0.6072,1
REACTION_TIME,0,0,1,1,0.321,1
MANIPULATION,1,1,0,1,1,0
VISION,1,1,0.008,0.0108,1,1
LANGUAGE,1,1,1,0.229091,0.373846,0.61875


In [20]:
def compute_sim_star(fw, fdrs):
    stars = pd.DataFrame("", index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            fdr = fdrs.loc[k,r]
            if fdr < 0.05:
                stars.loc[k,r] = "*"
            if fdr < 0.01:
                stars.loc[k,r] = "**"
            if fdr < 0.001:
                stars.loc[k,r] = "***"
    return stars

In [21]:
stars = compute_sim_star("rdoc", fdrs)
stars

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,***,,,***,***,
REWARD,,***,,,,
REACTION_TIME,***,***,,,,
MANIPULATION,,,***,,,***
VISION,,,**,*,,
LANGUAGE,,,,,,


## Weights for figure

In [22]:
sims[fdrs < 0.05] * 40

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,12.043,NaN,NaN,10.7216,11.6854,NaN
REWARD,NaN,16.1905,NaN,NaN,NaN,NaN
REACTION_TIME,16.8,20.8333,NaN,NaN,NaN,NaN
MANIPULATION,NaN,NaN,21.2766,NaN,NaN,24.2697
VISION,NaN,NaN,9.46237,9.3617,NaN,NaN
LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN


# Similarity of DSM and data-driven systems

## Observed values

In [23]:
sims = compute_sim_obs("dsm")
sims

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,0,0,0.08,0.0714286,0.245614,0,0.352941,0,0
REWARD,0,0,0.0444444,0,0,0,0,0.0851064,0.0869565
REACTION_TIME,0,0,0.105263,0.0952381,0.125,0.178571,0.0344828,0.20339,0.0689655
MANIPULATION,0.289157,0.301887,0.0416667,0,0,0,0,0,0.0408163
VISION,0.292683,0,0,0,0,0,0,0,0
LANGUAGE,0.206897,0.105263,0,0,0,0.0392157,0,0,0


## Null distribution

In [24]:
sims_null = compute_sim_null("dsm")

Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900


## False discovery rate

In [25]:
fdrs = compute_sim_fdr("dsm", sims, sims_null)
fdrs

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,0.999,0.947647,0.520714,0.888,0,0.947647,0,0.947647,0.947647
REWARD,0.999,0.947647,0.888,0.947647,0.947647,0.947647,0.947647,0.520714,0.520714
REACTION_TIME,0.999,0.947647,0.336,0.5832,0.3618,0.00771429,0.947647,0,0.837
MANIPULATION,0,0,0.947647,0.947647,0.947647,0.947647,0.947647,0.947647,0.947647
VISION,0,0.947647,0.947647,0.947647,0.947647,0.947647,0.947647,0.947647,0.947647
LANGUAGE,0.11475,0.520714,0.947647,0.947647,0.947647,0.947647,0.947647,0.947647,0.947647


In [26]:
stars = compute_sim_star("dsm", fdrs)
stars

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,,,,,***,,***,,
REWARD,,,,,,,,,
REACTION_TIME,,,,,,**,,***,
MANIPULATION,***,***,,,,,,,
VISION,***,,,,,,,,
LANGUAGE,,,,,,,,,


## Weights for figure

In [27]:
sims[fdrs < 0.05] * 40

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,NaN,NaN,NaN,NaN,9.82456,NaN,14.1176,NaN,NaN
REWARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
REACTION_TIME,NaN,NaN,NaN,NaN,NaN,7.14286,NaN,8.13559,NaN
MANIPULATION,11.5663,12.0755,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VISION,11.7073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
